In [82]:
import pandas as pd
from fpdf import FPDF
import yfinance as yf

# Load the Excel file
df_xlsx = pd.read_excel("Book2.xlsx")

# Filter buy and sell data
data_for_buy = df_xlsx[df_xlsx['TYPE'] == 'Buy']
data_for_sell = df_xlsx[df_xlsx['TYPE'] == 'Sell']

# Summarize the quantities by ticker
buy_summary = data_for_buy.groupby('EXCHANGE CODE: TICKER')['UNITS'].sum().to_dict()
sell_summary = data_for_sell.groupby('EXCHANGE CODE: TICKER')['UNITS'].sum().to_dict()
buy_price_dict = data_for_buy.groupby('EXCHANGE CODE: TICKER')['PRICE'].sum().to_dict()

# Function to fetch the current stock price using yfinance for Indian stocks
def get_stock_price(ticker):
    try:
        # Assuming NSE symbols, append '.NS' to the ticker. Adjust if using BSE.
        stock = yf.Ticker(f"{ticker[4:]}.NS")  # Use '.BO' for BSE
        price = stock.history(period="1d")['Close'].iloc[-1]  # Get the latest closing price
        return round(float(price), 2)
    except Exception as e:
        print(f"Error fetching price for {ticker[4:]}: {e}")
        return None

# Calculate the remaining stock and get the current price
result = {}
for key in sell_summary:
    buy_quantity = buy_summary.get(key, 0)
    sell_quantity = sell_summary[key]
    remaining_quantity = buy_quantity - sell_quantity
    buy_price = buy_price_dict.get(key, 0)
    current_price = get_stock_price(key)
    
    if current_price is not None:
        unrealized_profit = (current_price - buy_price) * remaining_quantity
        current_value = round(remaining_quantity * current_price, 2)
        total_cost_of_remaining_shares = buy_price * remaining_quantity
        profit_percentage = (unrealized_profit / total_cost_of_remaining_shares) * 100 if total_cost_of_remaining_shares > 0 else 0
        if int(current_value) == -0:
            current_value = 0 
        if int(remaining_quantity)==-0:
            remaining_quantity = 0
        if int(unrealized_profit) == -0:
            current_value = 0 
        if int(profit_percentage)==-0:
            remaining_quantity = 0
        result[key] = [
            round(remaining_quantity, 2),  # Remaining quantity
            current_price,  # Current price of the stock
            round(unrealized_profit, 2),  # Unrealized profit
            current_value,  # Current value of the portfolio/investment
            round(profit_percentage, 2)  # Profit percentage
        ]

# Print the result
print(result)
print(len(result.keys()))

{'NSE:AARTIIND': [0, 734.65, 0.0, 0, 0], 'NSE:ABBOTINDIA': [0, 26980.65, -35865.48, 18346.84, -66.16], 'NSE:AETHER': [0, 880.7, -0.0, 0, 0], 'NSE:AKZOINDIA': [0, 3312.45, 0.0, 0, 0], 'NSE:ALKEM': [0, 5714.85, 0.0, 0, 0], 'NSE:ALKYLAMINE': [0, 2056.3, 0.0, 0, 0], 'NSE:ATUL': [0, 7861.05, 0.0, 0, 0], 'NSE:BAJEL': [46.35, 279.3, 2192.36, 12945.55, 20.39], 'NSE:BANKBARODA': [0, 244.85, 0.0, 0, 0], 'NSE:BBTC': [0, 2310.1, -6.93, -11.55, 0], 'NSE:BIKAJI': [30.16, 861.85, -30730.93, 25993.4, -54.18], 'NSE:BSOFT': [0, 587.3, 0.0, 0, 0], 'NSE:CCL': [0, 654.6, 0.0, 0, 0], 'NSE:CYIENT': [0, 1665.05, 0.0, 0, 0], 'NSE:DCMSHRIRAM': [0, 1135.05, 0.0, 0, 0], 'NSE:DIXON': [0, 11664.25, 0.0, 0, 0], 'NSE:DLF': [0, 835.45, -0.0, 0, 0], 'NSE:ELGIEQUIP': [0, 621.1, 0.0, 0, 0], 'NSE:FEDERALBNK': [0, 201.24, 0.0, 0, 0], 'NSE:GRSE': [0, 2009.15, 0.0, 0, 0], 'NSE:HCLTECH': [14.04, 1585.25, -41231.97, 22256.91, -64.94], 'NSE:HEROMOTOCO': [0, 5311.85, 0.0, 0, 0], 'NSE:HGINFRA': [0, 1609.95, 0.0, 0, 0], 'NSE:HINDU

In [83]:
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=0)
pdf.add_page()

# Add a background color for the title
pdf.set_fill_color(200, 220, 255)
pdf.set_font("Arial", "B", 18)
pdf.cell(0, 12, txt="Stock Portfolio Summary", ln=True, align="C", fill=True)
pdf.ln(10)
pdf.set_right_margin(40)
# Set column headers with a background color and border
pdf.set_fill_color(169, 169, 169)  # Dark gray
pdf.set_text_color(255, 255, 255)  # White text for headers
pdf.set_font("Arial", "B", 12)
pdf.cell(38, 10, txt="Stock", border=1, align="C", fill=True)
pdf.cell(38, 10, txt="Total Quantities", border=1, align="C", fill=True)
pdf.cell(28, 10, txt="Current Price", border=1, align="C", fill=True)
pdf.cell(28, 10, txt="Profit/Loss", border=1, align="C", fill=True)
pdf.cell(28, 10, txt="Profit/Loss%", border=1, align="C", fill=True)
pdf.cell(38, 10, txt="Current Value", border=1, align="C", fill=True)
pdf.ln()

# Add data rows
pdf.set_font("Arial", "", 12)
pdf.set_text_color(0, 0, 0)  # Black text for data

for idx, (key, value) in enumerate(result.items()):
    # Alternate row background color for better readability
    if idx % 2 == 0:
        pdf.set_fill_color(245, 245, 245)  # Light gray
        fill = True
    else:
        fill = False

    # Add stock name and quantities
    pdf.cell(38, 10, txt=key[4:], border=1, align="C", fill=fill)  # Stock name without the exchange prefix
    pdf.cell(38, 10, txt=str(value[0]), border=1, align="C", fill=fill)  # Total quantities
    
    # Add current price
    pdf.cell(28, 10, txt=str(value[1]), border=1, align="C", fill=fill)  # Current price

    # Add profit with color coding
    if value[2] == 0:
        pdf.set_text_color(0, 128, 0)  # Green for profit
    else:
        pdf.set_text_color(255, 0, 0)  # Red for loss
    pdf.cell(28, 10, txt=str(value[2]), border=1, align="C", fill=fill)  # Profit
    pdf.set_text_color(0, 0, 0)  # Reset text color to black
    
    
    # Add profit percentage and current value
    pdf.cell(28, 10, txt=str(value[4]) + '%', border=1, align="C", fill=fill)  # Profit percentage
    pdf.cell(38, 10, txt=str(value[3]), border=1, align="C", fill=fill)  # Current value
    pdf.ln()

# Add a footer
pdf.set_y(-15)
pdf.set_font("Arial", "I", 8)
pdf.set_text_color(128, 128, 128)
pdf.cell(0, 10, txt="End of Report", align="C")

# Save the PDF to a file
pdf.output("stock_portfolio_summary.pdf")

print("PDF created successfully.")

PDF created successfully.
